In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 31 14:33:57 2023

@author: Lin
"""

import os
current_path = os.getcwd()
import numpy as np
from numpy import seterr
seterr(all='raise')
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import multiprocess as mp
import sys
sys.path.append(current_path+'/../../')
import myInput
import post_processing
import PACKAGE_MP_3DLinear as linear3d

import importlib
importlib.reload(post_processing)



<module 'post_processing' from '/home/lin.yang/projects/VECTOR/examples/dump_to_init/../../post_processing.py'>

In [2]:
def get_line(i, j):
    """Get the row order of grain i and grain j in MisoEnergy.txt (i < j)"""
    if i < j: return i+(j-1)*(j)/2
    else: return j+(i-1)*(i)/2

def init2EAarray(init_file_path, grain_num):
    euler_angle_array = np.ones((grain_num, 3))*-2
    with open(init_file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i > 2: euler_angle_array[int(line.split()[1])-1] = np.array([float(line.split()[2]), float(line.split()[3]), float(line.split()[4])])
    # Check if missing grains
    for i in range(grain_num):
        if euler_angle_array[i,0] == -2:
            print("Missing grains here!")
            euler_angle_array[i] = np.array([0,0,0])
    return euler_angle_array

def output_init_from_dump(dump_file_path, euler_angle_array, init_file_path_output):
    # output the init file with euler_angle_array and one dump file
    # Read necessary information from dump file
    with open(dump_file_path) as file:
        box_size = np.zeros(3)
        for i, line in enumerate(file):
            if i==3: num_sites = int(line)
            if i==5: box_size[0] = np.array(line.split(), dtype=float)[-1]
            if i==6: box_size[1] = np.array(line.split(), dtype=float)[-1]
            if i==7: box_size[2] = np.array(line.split(), dtype=float)[-1]
            if i==8: name_vars = line.split()[2:]
            if i>8: break
    box_size = np.ceil(box_size).astype(int) #reformat box_size
    entry_length = num_sites+9 #there are 9 header lines in each entry


    # write the IC files and read the dump data
    # create IC file
    IC_nei = []
    IC_nei.append("# This line is ignored\n")
    IC_nei.append("Values\n")
    IC_nei.append("\n")
    with open(init_file_path_output, 'w') as output_file:
        output_file.writelines( IC_nei )
    IC_nei = []
    # read and write
    with open(init_file_path_output, 'a') as output_file:
        with open(dump_file_path) as file:
            for i, line in tqdm(enumerate(file), "EXTRACTING SPPARKS DUMP (%s.dump)"%dump_file_path[-20:], total=entry_length):
                if i==1: time_step = int(float(line.split()[-1])) #log the time step
                atom_num = i-9 #track which atom line we're on
                if atom_num>=0 and atom_num<num_sites:
                    line_split = np.array(line.split(), dtype=float)
                    grain_id = int(line_split[1])-1
                    output_file.write(f"{int(line_split[0])} {int(line_split[1])} {euler_angle_array[grain_id, 0]} {euler_angle_array[grain_id, 1]} {euler_angle_array[grain_id, 2]}\n")

    return box_size, entry_length

def output_init_neighbor_from_init(interval, box_size, init_file_path_input, init_file_path_output):
    # Output the init_nighbor5 with init file

    nei_num = (2*interval+3)**3-1
    size_x,size_y,size_z = box_size
    img = np.zeros((size_y,size_x,size_z)) #Figure of all sites with GrainID

    print(f"> img matrix start.")
    for k in tqdm(range(size_z)): # z-axis
        for i in range(size_y): # y-axis
            for j in range(size_x): # x-axis
                img[i,j,k] = int(k*size_x*size_y + i*size_x + j)
    print(f"> img matrix end")

    IC_nei = []
    IC_nei.append("# This line is ignored\n")
    IC_nei.append("3 dimension\n")
    IC_nei.append(f"{nei_num} max neighbors\n")
    IC_nei.append(f"{size_x*size_y*size_z} sites\n")
    IC_nei.append(f"0 {size_x} xlo xhi\n")
    IC_nei.append(f"0 {size_y} ylo yhi\n")
    IC_nei.append(f"0 {size_z} zlo zhi\n")
    IC_nei.append("\n")
    IC_nei.append("Sites\n")
    IC_nei.append("\n")
    with open(init_file_path_output, 'w') as file:
        file.writelines( IC_nei )
    IC_nei = []

    print("> Sites start writing")
    with open(init_file_path_output, 'a') as file:
        for k in tqdm(range(size_z)): # z-axis
            for i in range(size_y): # y-axis
                for j in range(size_x): # x-axis
                    file.write(f"{int(img[i,j,k] + 1)} {float(j)} {float(i)} {float(k)}\n")
    print("> Sites end writing")

    IC_nei.append("\n")
    IC_nei.append("Neighbors\n")
    IC_nei.append("\n")
    with open(init_file_path_output, 'a') as file:
        file.writelines( IC_nei )
    IC_nei = []

    # offset value setting before neighboring start
    offsets = np.array(np.meshgrid(
    np.arange(-(interval + 1), interval + 2),
    np.arange(-(interval + 1), interval + 2),
    np.arange(-(interval + 1), interval + 2),
    )).T.reshape(-1, 3)
    # Filter out the [0, 0, 0] offset since we want to skip it
    offsets = offsets[np.any(offsets != 0, axis=1)]

    def process_chunk(start_k, end_k, file_name):
        max_length_neighbors = 0
        with open(file_name, 'w') as file:
            for k in tqdm(range(start_k, end_k)): # z-axis
                for i in range(size_y): # y-axis
                    for j in range(size_x): # x-axis
                        tmp_nei = f"{int(img[i,j,k] + 1)} "
                        # Compute the indices with wrapping around boundaries (using np.mod)
                        indices = (np.array([i, j, k]) + offsets) % np.array([size_y, size_x, size_z])
                        # Extract the values from 'img' using advanced indexing
                        neighbour_values = img[indices[:, 0], indices[:, 1], indices[:, 2]].astype('int')
                        # Convert values to 1-based indexing and concatenate into a string
                        tmp_nei += ' '.join(map(str, neighbour_values + 1))
                        max_length_neighbors = max(max_length_neighbors, len(tmp_nei))
                        file.write(tmp_nei + "\n")
            file.write("\n")
        print(f"The max length of neighbor data line is {max_length_neighbors}")

    print("> Neighbors start writing")
    # ecessary init for multi cores writing
    num_processes = mp.cpu_count() # or choose a number that suits your machine
    chunk_size = size_z // num_processes
    processes = []
    temp_files = []
    # Assign tasks for processors
    for p in range(num_processes):
        start_k = p * chunk_size
        end_k = (p + 1) * chunk_size if p != num_processes - 1 else size_z
        temp_file = f'{init_file_path_output}_temp_{p}.txt'
        temp_files.append(temp_file)
        process = mp.Process(target=process_chunk, args=(start_k, end_k, temp_file))
        processes.append(process)
        process.start()
    # Wait for all processes to complete
    for process in processes:
        process.join()
    # Concatenate all temporary files
    with open(init_file_path_output, 'a') as outfile:
        for fname in tqdm(temp_files, "Concatenating "):
            with open(fname) as infile:
                outfile.write(infile.read())
            os.remove(fname)  # Optional: remove temp file after concatenation

    print("> Neighbors end writing")

    print("> Values start writing")
    with open(init_file_path_input, 'r') as f_read:
        tmp_values = f_read.readlines()
    print("> Values read done")
    with open(init_file_path_output, 'a') as file:
        file.writelines(tmp_values[1:])
    print("> Values end writing")
    return True



In [3]:



if __name__ == '__main__':


    # File name
    # dump file of last time step
    last_step = 5
    # dump_file_foler = "/orange/michael.tonks/lin.yang/"
    dump_file_foler = "/blue/michael.tonks/lin.yang/SPPARKS-VirtualIncEnergy/2d_poly_multiCoreCompare/"
    # dump_file_foler = "/Users/lin/projects/SPPARKS-AGG/examples/Test_SimplifyIncE/2d_triple/"
    # dump_file_name = f"p_ori_fully5d_aveE_f1.0_t1.0_150_1k_multiCore64_J1_refer_1_0_0_seed56689_kt1.95"
    init_file_folder = dump_file_foler + "IC/"
    init_file_name = f"oneGB_512.init"
    init_file_name_final = f"oneGB_512_neighbors5.init"
    # init_file_name_final = f"poly_IC150_1k.{last_step}_neighbor5.init"

    # Read init file
    grain_num = 2
    # euler_angle_array = post_processing.init2EAarray(init_file_folder+init_file_name_original, grain_num)


    # Read necessary information from dump file
    # dump_file_name_0 = dump_file_foler+dump_file_name+f".dump.{int(last_step)}"
    # box_size, entry_length = post_processing.output_init_from_dump(dump_file_name_0, euler_angle_array, init_file_folder+init_file_name)
    # size_x,size_y,size_z = box_size



In [4]:

if __name__ == '__main__':

  # necessary data for neighbor init file
  interval = 5
  box_size = np.array([512,512,1])
  output_neighbr_init = post_processing.output_init_neighbor_from_init_mp(interval, box_size, init_file_folder+init_file_name, init_file_folder+init_file_name_final)
  # output_neighbr_init = output_init_neighbor_from_init(interval, box_size, init_file_folder+init_file_name, init_file_folder+init_file_name_final+"_test")



> img matrix start.
> img matrix end
> Sites start writing


100%|██████████| 1/1 [00:00<00:00,  2.15it/s]

> Sites end writing
> Neighbors start writing



100%|██████████| 4/4 [00:00<00:00, 33.48it/s]


The max length of neighbor data line is 987


...

The max length of neighbor data line is 1182

100%|██████████| 4/4 [00:00<00:00, 28.61it/s]

The max length of neighbor data line is 1182
The max length of neighbor data line is 1091
The max length of neighbor data line is 1182
The max length of neighbor data line is 1182
The max length of neighbor data line is 1182The max length of neighbor data line is 1182

The max length of neighbor data line is 1182
The max length of neighbor data line is 1182
The max length of neighbor data line is 1182


Concatenating : 100%|██████████| 128/128 [00:00<00:00, 231.36it/s]


> Neighbors end writing
> Values start writing
> Values read done
> Values end writing
